In [1]:
using Pkg
using JuMP
using Gurobi
#using GLPK
using XLSX
using DataFrames
using CSV

In [2]:
using LinearAlgebra

In [3]:
shared_cand = DataFrame(XLSX.readtable("CandLine.xlsx", "Taul1", header=true)...) #Dataframe of shared candidate lines
int_cand =  DataFrame(XLSX.readtable("CandLineInt.xlsx", "Taul1", header=true)...) #Dataframe of internal candidate lines
shared_ex =  DataFrame(XLSX.readtable("SharedEline.xlsx", "Taul1", header=true)...) #Dataframe of shared existing lines
int_ex = DataFrame(XLSX.readtable("Tran.xlsx", "Taul1", header=true)...) #Dataframe of internal existing lines
gen =  DataFrame(XLSX.readtable("Gen.xlsx" , "Taul1", header=true)...) #Dataframe of generators
load =  DataFrame(XLSX.readtable("Load.xlsx", "Taul1", header=true)...) #Dataframe of loads
scen_prob = DataFrame(XLSX.readtable("Scenario_Probability.xlsx", "Taul1", header=true)...)
zone_summary = DataFrame(CSV.File("Zone_Summary.csv", header=true)) #Dataframe of region-number of nodes
##An example of the load dataframe can be found as follows

,Zone,Nodes_Total
,Int64,Int64
1,1,14
2,2,30
3,3,5


In [4]:
l(i,s) = load[load.zoneNum .== i,[1,2,s+2]] # load within zone i and scenario s
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1 .* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i
scen_weight(s) = scen_prob.scen_weight[scen_prob.scenario .== s,:]

scen_weight (generic function with 1 method)

In [5]:
S = nrow(scen_prob)

4

In [6]:
function bin(i,N,S)
    P = zeros(nrow(zone_summary),maximum(zone_summary.Nodes_Total),S); 
    P[i,1:N,:] .= 1
    for c in 1:nrow(shared_c(i))
        P[shared_c(i).nodeZone1[c],shared_c(i).tNodeID1[c],:] .= 1
        P[shared_c(i).nodeZone2[c],shared_c(i).tNodeID2[c],:] .= 1
    end
    for h in 1:nrow(shared_e(i))
        P[shared_e(i).nodeZone1[h],shared_e(i).tNodeID1[h],:] .= 1
        P[shared_e(i).nodeZone2[h],shared_e(i).tNodeID2[h],:] .= 1
    end
    return(P)
end

bin (generic function with 1 method)

In [7]:
bin(1,14,S)

3×30×4 Array{Float64, 3}:
[:, :, 1] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0

In [8]:
function zonal_app(i, N, S, lambda, shared_line_decision_var, int_line_decision_var, angle_self, angle_1, angle_2)
    
    Mod4 = Model(Gurobi.Optimizer)
    
    
    @variable(Mod4,0 <= gen_var[1:N,1:S]) # generation at each node
    @variable(Mod4,shared_cand_flow[1:nrow(shared_cand),1:S])  #Power flowing into shared candidate lines 
    @variable(Mod4,int_cand_flow[1:nrow(int_c(i)), 1:S]) #Power flowing into internal candidate lines 
    #@variable(Mod4,0 <= shared_cand_angle[1:nrow(shared_cand), 1:S, 1:2]<= 2*pi) #Phase angle decision for shared candidate lines
    #@variable(Mod4,0 <= int_cand_angle[1:nrow(int_c(i)), 1:S, 1:2]<= 2*pi) #Phase angle decision for internal candidate lines
    @variable(Mod4,shared_ex_flow[1:nrow(shared_ex), 1:S])  #Power flowing on shared existing shared lines
    @variable(Mod4,int_ex_flow[1:nrow(int_e(i)), 1:S])  #Power flowing on internal existing lines
    @variable(Mod4,0 <= node_voltage_phase_angle[1:nrow(zone_summary),1:maximum(zone_summary.Nodes_Total), 1:S] <= 2*pi)
   # @variable(Mod4,0 <= shared_ex_angle[1:nrow(shared_ex), 1:S, 1:2]<= 2*pi) #Phase angle for existing shared lines
    #@variable(Mod4,0 <= int_ex_angle[1:nrow(int_e(i)), 1:S, 1:2]<= 2*pi) #Phase angle for internal existing lines
    
    omega = 0.25;
    eta = 0.5;
    
    @expression(Mod4, total_cost, sum(scen_weight(s)[1].* sum((gen_var[n,s] .* sum((g(i).gNodeID .== n) .* MC(i))) for n in 1:N)
        .+ sum(sum(eta * node_voltage_phase_angle[z,m,s] .* (angle_self[z,m,s] .- angle_1[z,m,s])
        .+ eta * node_voltage_phase_angle[z,m,s] .* (angle_self[z,m,s] .- angle_2[z,m,s])
        .+ (omega /2) * (node_voltage_phase_angle[z,m,s] .- angle_self[z,m,s]) .^ 2
        .+ (omega /2) * (node_voltage_phase_angle[z,m,s] .- angle_1[z,m,s]) .^ 2
        .+ (omega /2) * (node_voltage_phase_angle[z,m,s] .- angle_2[z,m,s]) .^ 2
        .+ node_voltage_phase_angle[z,m,s] .*
        lambda[z,m,s] for m in 1:maximum(zone_summary.Nodes_Total)) for z in 1:nrow(zone_summary)) for s in 1:S))
            
            
    for s in 1:S        
            
        for n in 1:N
            # Power balance constraint for each node
            @constraint(Mod4, sum((g(i).gNodeID .== n) .* gen_var[n,s]) .+ sum(l(i,s)[:,3].* (l(i,s).lNodeID .== n))./100 .==
                sum((shared_cand.tNodeID1 .== n) .*bin_c(i) .* shared_cand_flow[:,s]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(i) .* shared_cand_flow[:,s]) .+
                sum((shared_ex.tNodeID1 .== n) .* bin_e(i) .* shared_ex_flow[:,s]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(i) .* shared_ex_flow[:,s]) .+
                sum((int_c(i).tNodeID1 .== n) .* int_cand_flow[:,s]) .- sum((int_c(i).tNodeID2 .== n) .* int_cand_flow[:,s]) .+
                sum((int_e(i).tNodeID1 .== n) .* int_ex_flow[:,s]) .- sum((int_e(i).tNodeID2 .== n) .* int_ex_flow[:,s]))
            #Lower limit for generation of each node
            @constraint(Mod4, sum(g(i).gNodeID .== n) .* gen_var[n,s] .<= sum((g(i).gNodeID .== n) .* g(i).PgMax./100))
            #Upper limit for generation of each node
            @constraint(Mod4, sum((g(i).gNodeID .== n) .* g(i).PgMin./100) .<= sum(g(i).gNodeID .== n) .* gen_var[n,s])
        end

        M = 4 #Big number

        #Constraints elated to the candidate shared lines:
        for c in 1:nrow(shared_cand) 
            #Associating the power flowing within the line with the phase angles of nodes:
            @constraint(Mod4,-M .* (1 .- bin_c(i) .* shared_line_decision_var[c]) .<= bin_c(i) .* shared_cand_flow[c,s] .- 
                ((1 ./ shared_cand.reacT[c]) .* bin_c(i) .* 
                (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s] 
                .- node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s])))
            @constraint(Mod4, bin_c(i) .* shared_cand_flow[c,s] .- ((1 ./ shared_cand.reacT[c]) .*
                (bin_c(i) .* (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s] .- 
                node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s]))) 
                .<= M .* (1 .- bin_c(i) .* shared_line_decision_var[c]))
            #limiting the upper bound of power flow flowing within candidate shared lines
            @constraint(Mod4, bin_c(i) .* shared_cand_flow[c,s] .<= bin_c(i) .* 
                shared_line_decision_var[c] .* shared_cand.ptMax[c] ./ 100)
            #Limiting the lower bound of power flowing within the candidate shared lines
            @constraint(Mod4, -shared_cand.ptMax[c] .* bin_c(i) .* shared_line_decision_var[c] ./ 100 .<= bin_c(i) .* 
                shared_cand_flow[c,s])
            #@constraint(Mod4, shared_cand_angle[5,2] .== 0)
        end

        #The same constraints related to the candidate internal lines:
        for c in 1:nrow(int_c(i))
            @constraint(Mod4,-M .* (1 .- int_line_decision_var[c]) .<= int_cand_flow[c,s] .- ((1 ./ int_c(i).reacT[c]) .*
                    (node_voltage_phase_angle[int_c(i).zoneNum[c],int_c(i).tNodeID1[c],s] .-
                    node_voltage_phase_angle[int_c(i).zoneNum[c],int_c(i).tNodeID2[c],s])))
            @constraint(Mod4, int_cand_flow[c,s] .- ((1 ./ int_c(i).reacT[c]) .* 
                    (node_voltage_phase_angle[int_c(i).zoneNum[c],int_c(i).tNodeID1[c],s] .- 
                    node_voltage_phase_angle[int_c(i).zoneNum[c],int_c(i).tNodeID2[c],s]))
                    .<= M .* (1 .- int_line_decision_var[c]))
            @constraint(Mod4, int_cand_flow[c,s] .<= int_line_decision_var[c] .* int_c(i).ptMax[c] ./ 100)
            @constraint(Mod4, -int_c(i).ptMax[c] .*int_line_decision_var[c] ./ 100 .<= int_cand_flow[c,s])
        end

        #The same constraints related to the shared existing lines:
        for h in 1:nrow(shared_ex)
            @constraint(Mod4, bin_e(i) .* shared_ex_flow[h,s] .== (1 ./ shared_ex.reacT[h]) .*
                (bin_e(i) .* (node_voltage_phase_angle[shared_ex.nodeZone1[h],shared_ex.tNodeID1[h],s] .-
                node_voltage_phase_angle[shared_ex.nodeZone2[h],shared_ex.tNodeID2[h],s])))
            @constraint(Mod4, bin_e(i) .* shared_ex_flow[h,s] .<= bin_e(i) .* shared_ex. ptMax[h] ./ 100)
            @constraint(Mod4, bin_e(i) .* -shared_ex.ptMax[h] ./ 100 .<= bin_e(i) .* shared_ex_flow[h,s])
            #@constraint(Mod4, shared_ex_angle[1,2] .== 0)
            #@constraint(Mod4, shared_ex_angle[7,2] .== 0)

        end
        #The same constraints related to the internal existing lines:

        for h in 1:nrow(int_e(i))
            @constraint(Mod4, int_ex_flow[h,s] .== (1 ./ int_e(i).reacT[h]) 
                .* (node_voltage_phase_angle[int_e(i).zoneNum[h],int_e(i).tNodeID1[h],s] .- 
                    node_voltage_phase_angle[int_e(i).zoneNum[h],int_e(i).tNodeID2[h],s]))
            @constraint(Mod4, int_ex_flow[h,s] .<= int_e(i).ptMax[h] ./ 100)
            @constraint(Mod4, -int_e(i).ptMax[h] ./ 100 .<= int_ex_flow[h,s])
        end
    end
    #The model objective is to minimize the total cost:
    
    @objective(Mod4, Min, total_cost)
    optimize!(Mod4)
    if termination_status(Mod4) == MOI.OPTIMAL #If the solution is optimal then it returns the variables
    
        shared_cand_power = value.(shared_cand_flow)
        shared_ex_power = value.(shared_ex_flow)
        phase_angle = value.(node_voltage_phase_angle)
        gen_power = value.(gen_var)
        int_cand_power = value.(int_cand_flow)
        obj_value = objective_value(Mod4)
        [obj_value,shared_cand_power,phase_angle,shared_ex_power,gen_power, int_cand_power]

    end
end
    
    

zonal_app (generic function with 1 method)

In [9]:
f=ones(12000)
h=0
lambda_1= zeros(12000,nrow(zone_summary),maximum(zone_summary.Nodes_Total),S)
lambda_2= zeros(12000,nrow(zone_summary),maximum(zone_summary.Nodes_Total),S)
lambda_3 = zeros(12000,nrow(zone_summary),maximum(zone_summary.Nodes_Total),S)

angle_1 = zeros(12000,nrow(zone_summary),maximum(zone_summary.Nodes_Total),S)
angle_2 = zeros(12000,nrow(zone_summary),maximum(zone_summary.Nodes_Total),S)
angle_3 = zeros(12000,nrow(zone_summary),maximum(zone_summary.Nodes_Total),S)

cand_flow_1 =zeros(12000,nrow(shared_cand),S)
cand_flow_2 =zeros(12000,nrow(shared_cand),S)
cand_flow_3 =zeros(12000,nrow(shared_cand),S)

ex_flow_1 = zeros(12000,nrow(shared_ex),S)
ex_flow_2 = zeros(12000,nrow(shared_ex),S)
ex_flow_3 = zeros(12000,nrow(shared_ex),S)

int_cand_flow_1 = zeros(12000,nrow(int_c(1)),S)
int_cand_flow_2 = zeros(12000,nrow(int_c(2)),S)
int_cand_flow_3 = zeros(12000,nrow(int_c(3)),S)

gen_1 = zeros(12000,zone_summary.Nodes_Total[1],S)
gen_2 = zeros(12000,zone_summary.Nodes_Total[2],S)
gen_3 = zeros(12000,zone_summary.Nodes_Total[3],S)

iter = 0



shared_line_decision_var_m =  
[0.0
 0.0
 0.0
 1.0
 1.0
 0.0]
int_line_decision_var_1 =
[0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0]
int_line_decision_var_2 =
[1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0]
int_line_decision_var_3 =
[0.0
 0.00]

2-element Vector{Float64}:
 0.0
 0.0

In [10]:
size(lambda_2)

(12000, 3, 30, 4)

In [11]:
    j = 2
    max_iter = 500
    for j in 2:max_iter    # It considers 1000 iteration
        if abs(f[j-1]) > 0.01  
         
            R_1 = zonal_app(1, 14, S, lambda_1[j,:,:,:], shared_line_decision_var_m, int_line_decision_var_1,
            bin(1,14,S) .* angle_1[j-1,:,:,:], bin(1,14,S) .* bin(2,30,S) .* angle_2[j-1,:,:,:],  bin(1,14,S) .* bin(3,5,S) .* angle_3[j-1,:,:,:])
        
            R_2 = zonal_app(2, 30, S, lambda_2[j,:,:,:], shared_line_decision_var_m, int_line_decision_var_2,
            bin(2,30,S) .* angle_2[j-1,:,:,:],  bin(2,30,S) .* bin(1,14,S) .* angle_1[j-1,:,:,:],   bin(2,30,S) .* bin(3,5,S) .* angle_3[j-1,:,:,:])
        
            R_3 = zonal_app(3, 5, S, lambda_3[j,:,:,:], shared_line_decision_var_m, int_line_decision_var_3,
             bin(3,5,S) .* angle_3[j-1,:,:,:], bin(3,5,S) .* bin(1,14,S) .* angle_1[j-1,:,:,:], bin(3,5,S) .* bin(2,30,S) .* angle_2[j-1,:,:,:])
        
            
        
        
            lambda_1[j+1,:,:,:] .= bin(1,14,S) .* lambda_1[j,:,:,:] .+ 0.01 * bin(1,14,S) .* (R_1[3] .- (bin(2,30,S) .* R_2[3] .+ bin(3,5,S) .* R_3[3]))
            lambda_2[j+1,:,:,:] .= bin(2,30,S) .* lambda_2[j,:,:,:] .+ 0.01 * bin(2,30,S) .* (R_2[3] .- (bin(1,14,S) .* R_1[3] .+ bin(3,5,S) .* R_3[3]))
            lambda_3[j+1,:,:,:] .= bin(3,5,S) .* lambda_3[j,:,:,:] .+ 0.01 * bin(3,5,S).* (R_3[3] .- (bin(1,14,S) .* R_1[3] .+ bin(2,30,S) .* R_2[3]))
        
        
        
            angle_1[j,:,:,:] = R_1[3]
            angle_2[j,:,:,:] = R_2[3]
            angle_3[j,:,:,:] = R_3[3]
            
            cand_flow_1[j,:,:] = R_1[2] .* 100
            cand_flow_2[j,:,:] = R_2[2] .* 100
            cand_flow_3[j,:,:] = R_3[2] .* 100
        
            ex_flow_1[j,:,:] = R_1[4] .* 100
            ex_flow_2[j,:,:] = R_2[4] .* 100
            ex_flow_3[j,:,:] = R_3[4] .* 100
        
            gen_1[j,:,:] = R_1[5] .* 100
            gen_2[j,:,:] = R_2[5] .* 100
            gen_3[j,:,:] = R_3[5] .* 100
        
            
            f[j] = sum(sum(bin(1,14,S) .* (R_1[3] .- (bin(2,30,S) .* R_2[3] .+ bin(3,5,S) .* R_3[3])))) +
                    sum(sum(bin(2,30,S) .* (R_2[3] .- (bin(1,14,S) .* R_1[3] .+ bin(3,5,S) .* R_3[3])))) +
                    sum(sum(bin(3,5,S).* (R_3[3] .- (bin(1,14,S) .* R_1[3] .+ bin(2,30,S) .* R_2[3]))))

            h= R_2[1] + R_1[1] + R_3[1]
            
            if isdefined(Main, :IJulia)
                Main.IJulia.stdio_bytes[] = 0
            end
        
            print('*') 
        
        else
            iter = j-1
            print(iter)
            break
        end
    end
    

Academic license - for non-commercial use only - expires 2023-06-28
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1912 rows, 592 columns and 2608 nonzeros
Model fingerprint: 0xc00b9525
Model has 360 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [9e+00, 1e+01]
  QObjective range [8e-01, 8e-01]
  Bounds range     [6e+00, 6e+00]
  RHS range        [9e-03, 4e+00]
Presolve removed 1692 rows and 368 columns
Presolve time: 0.00s
Presolved: 220 rows, 276 columns, 708 nonzeros
Presolved model has 104 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.360e+02
 Factor NZ  : 3.208e+03
 Factor Ops : 5.560e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.93348023e+07

In [12]:
if iter == 0
    iter = max_iter
end

In [13]:
iter

10

In [14]:
h #OF of the second stage

17.580091298698488

In [15]:
f[iter]

-0.006659640028847669

In [16]:
angle_1[iter,:,:,:]

3×30×4 Array{Float64, 3}:
[:, :, 1] =
 0.143561  0.0992476   2.5482e-12  …  0.0         0.0  0.0  0.0  0.0  0.0
 0.0       0.0         0.0            5.36227e-5  0.0  0.0  0.0  0.0  0.0
 0.308972  1.2303e-10  5.47546e-5     0.0         0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0704666    0.0465801    7.65235e-12  …  0.0        0.0  0.0  0.0  0.0  0.0
 0.0          0.0          0.0             5.4038e-5  0.0  0.0  0.0  0.0  0.0
 5.26378e-11  1.73823e-10  5.47546e-5      0.0        0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0932179  0.069658     2.91827e-12  …  0.0         0.0  0.0  0.0  0.0  0.0
 0.0        0.0          0.0             5.37316e-5  0.0  0.0  0.0  0.0  0.0
 0.301218   1.52474e-10  5.47546e-5      0.0         0.0  0.0  0.0  0.0  0.0

[:, :, 4] =
 0.0352382    0.023305    1.52979e-11  …  0.0         0.0  0.0  0.0  0.0  0.0
 0.0          0.0         0.0             5.41417e-5  0.0  0.0  0.0  0.0  0.0
 8.89303e-11  1.30664e-9  5.47546e-5      0.0         0.0  0.0  0.0  0.0  0.0

In [17]:
angle_2[iter,:,:,:]

3×30×4 Array{Float64, 3}:
[:, :, 1] =
 0.0704844   4.54368e-5  0.0         …  0.0        0.0       0.0
 0.0623405   0.0408161   0.00986238     0.0134675  0.039379  0.0227668
 4.44154e-5  0.307164    0.0979004      0.0        0.0       0.0

[:, :, 2] =
 4.51771e-5  4.54406e-5  0.0        …  0.0         0.0         0.0
 0.0251646   0.0313373   0.0074302     0.00569697  0.00830608  6.05999e-11
 4.40595e-5  0.0977709   0.0217367     0.0         0.0         0.0

[:, :, 3] =
 0.0748224  4.54393e-5  0.0          …  0.0        0.0        0.0
 0.0377112  0.0280531   0.000780292     0.0119009  0.0492992  0.0368401
 4.4433e-5  0.311502    0.105637        0.0        0.0        0.0

[:, :, 4] =
 4.49815e-5  4.54419e-5  0.0        …  0.0        0.0         0.0
 0.0198803   0.0231933   0.010362      0.0073197  0.00415304  3.17302e-11
 4.39995e-5  0.0314588   0.0192488     0.0        0.0         0.0

In [18]:
angle_3[iter,:,:,:]

3×30×4 Array{Float64, 3}:
[:, :, 1] =
 0.0548068  0.00750216  0.0        0.0          …  0.0  0.0  0.0  0.0  0.0
 0.0        0.0         0.0        0.0886339       0.0  0.0  0.0  0.0  0.0
 0.0136592  0.00303244  0.0014116  2.32665e-12     0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 1.88455e-9  3.04154e-9  0.0          0.0          …  0.0  0.0  0.0  0.0  0.0
 0.0         0.0         0.0          0.037837        0.0  0.0  0.0  0.0  0.0
 0.0147147   0.00406382  8.88602e-11  1.24785e-11     0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.108429     0.0287483    0.0          0.0        …  0.0  0.0  0.0  0.0  0.0
 0.0          0.0          0.0          0.0855315     0.0  0.0  0.0  0.0  0.0
 2.06786e-11  2.94977e-12  2.93184e-12  0.003         0.0  0.0  0.0  0.0  0.0

[:, :, 4] =
 3.89995e-9  9.4268e-9   0.0          …  0.0        0.0  0.0  0.0  0.0  0.0
 0.0         0.0         0.0             0.0221215  0.0  0.0  0.0  0.0  0.0
 0.0161181   0.00806262  0.000399579     0.0        0.0  0.0  0.0  0.0  0.0

In [19]:
cand_flow_1[iter,:,:]

6×4 Matrix{Float64}:
    0.0   0.0         0.0   0.0
    0.0   0.0         0.0   0.0
    0.0   0.0         0.0   0.0
  100.0  90.9637    100.0  45.5108
 -100.0  -9.95409  -100.0  -6.50434
    0.0   0.0         0.0   0.0

In [20]:
cand_flow_2[iter,:,:]

6×4 Matrix{Float64}:
  0.0       0.0        0.0      0.0
  0.0       0.0        0.0      0.0
  0.0       0.0        0.0      0.0
 -4.82887  -0.485078  -1.06046  0.682197
  6.25364   2.67904    4.84196  3.64853
  0.0       0.0        0.0      0.0

In [21]:
cand_flow_3[iter,:,:]

6×4 Matrix{Float64}:
   0.0   0.0       0.0   0.0
   0.0   0.0       0.0   0.0
   0.0   0.0       0.0   0.0
   0.0   0.0       0.0   0.0
 100.0  46.5382  100.0  12.47
   0.0   0.0       0.0   0.0

In [22]:
ex_flow_1[iter,:,:]

8×4 Matrix{Float64}:
   54.2773      38.5363       51.3974      19.2392
 -100.0          3.90545    -100.0          4.44824
   79.5246     -33.8782      100.0        -16.9414
   72.2942      -4.64815      77.2005      -1.88068
  -58.1603      22.6852      -57.3094      11.2395
    0.0268113    0.0270189     0.0268657    0.0270702
 -100.0        -50.9474     -100.0          0.0495138
    0.0          0.0           0.0          0.0

In [23]:
ex_flow_2[iter,:,:]

8×4 Matrix{Float64}:
   0.262408     -8.77939        0.531976     -3.52583
  -2.89633      -0.708072      -3.32678       4.94551
 -33.8654       -0.000537311  -35.9509       -0.00047211
  -0.00837926   -0.00837953    -0.00837962   -0.00837942
  -8.3485e-5    -0.000158117   -8.0297e-5    -0.000184642
 -94.2689      -27.6469       -96.6498       -6.28835
  10.3064        6.31602        5.40308     -11.6961
  33.2572        8.26181       34.1782        6.83236

In [24]:
ex_flow_3[iter,:,:]

8×4 Matrix{Float64}:
   0.0           0.0        0.0           0.0
 100.0          65.1538   100.0          11.2468
 -19.7825        7.07436  -52.1295        7.74911
  -7.04891e-9   -2.12947    0.539568     -5.97747e-7
  22.8145       23.8825     1.86604e-8   19.9753
  -1.51622      -2.03191    1.12488e-10   7.02946
  70.8486       32.0165    63.2104        8.28115
 -43.7017      -14.1112   -23.75         -1.77528

In [25]:
gen_1[iter,:,:]

14×4 Matrix{Float64}:
 104.723        8.89998e-11  42.8526       8.86125e-11
   1.39614e-10  5.53914e-11   1.39356e-10  5.52075e-11
   0.0          0.0           0.0          0.0
   0.0          0.0           0.0          0.0
   0.0          0.0           0.0          0.0
   0.0          0.0           0.0          0.0
   0.0          0.0           0.0          0.0
   0.0          0.0           0.0          0.0
   0.0          0.0           0.0          0.0
   0.0          0.0           0.0          0.0
   0.0          0.0           0.0          0.0
   0.0          0.0           0.0          0.0
   0.0          0.0           0.0          0.0
   0.0          0.0           0.0          0.0

In [26]:
gen_2[iter,:,:]

30×4 Matrix{Float64}:
 69.2          1.18246e-8  39.1521       2.69103e-9
 80.0         80.0         80.0         47.3
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
 40.0         14.6         22.7479       3.88968e-9
  ⋮                                     
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  2.97862e-8   8.78082e-9   1.8792e-8    2.44964e-9
  1.29202e-8   5.45893e-9   9.76195e-9   2.09656e-9
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0    

In [27]:
gen_3[iter,:,:]

5×4 Matrix{Float64}:
 1.91177e-11  1.90882e-11  1.91229e-11  1.90813e-11
 8.42315e-12  8.41104e-12  8.42661e-12  8.40781e-12
 0.0          0.0          0.0          0.0
 0.0          0.0          0.0          0.0
 0.0          0.0          0.0          0.0

In [28]:
gen_2[iter,:,:]

30×4 Matrix{Float64}:
 69.2          1.18246e-8  39.1521       2.69103e-9
 80.0         80.0         80.0         47.3
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
 40.0         14.6         22.7479       3.88968e-9
  ⋮                                     
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  2.97862e-8   8.78082e-9   1.8792e-8    2.44964e-9
  1.29202e-8   5.45893e-9   9.76195e-9   2.09656e-9
  0.0          0.0          0.0          0.0
  0.0          0.0          0.0          0.0
  0.0    